Part 1 for the algorithm to simulate the problem we need to create the grid world then we solveing the bellman equations 

In [3]:
import numpy as np

# Define the gridworld environment
state_space = [(i, j) for i in range(5) for j in range(5)]
action_space = ['up', 'down', 'left', 'right']
num_states = len(state_space)
num_actions = len(action_space)

# Map states to indices and vice versa
state_to_idx = {state: idx for idx, state in enumerate(state_space)}
idx_to_state = {idx: state for state, idx in state_to_idx.items()}

# Define the special states
blue = (0, 1)
green = (0, 3)
red = (1, 3)
yellow = (4, 3)

# Define the transition probabilities and rewards
P = np.zeros((num_states, num_actions, num_states)) # P for transition probabilities
R = np.zeros((num_states, num_actions)) # R for rewards

def step(state, action):
    i, j = state
    if action == 'up':
        next_state = (max(i - 1, 0), j)
    elif action == 'down':
        next_state = (min(i + 1, 4), j)
    elif action == 'left':
        next_state = (i, max(j - 1, 0))
    elif action == 'right':
        next_state = (i, min(j + 1, 4))
    else:
        next_state = state

    # Special states and rewards
    if state == blue:
        reward = 5
        next_state = red
    elif state == green:
        reward = 2.5
        next_state = yellow if np.random.rand() < 0.5 else red
    elif next_state == state:
        reward = -0.5  # Stepping off the grid
    else:
        reward = 0  # Normal movement

    return next_state, reward

# Fill in the transition matrix P and reward vector R
for state in state_space:
    state_idx = state_to_idx[state] # Get the index of the current state
    for action_idx, action in enumerate(action_space):
        next_state, reward = step(state, action)
        next_state_idx = state_to_idx[next_state]
        P[state_idx, action_idx, next_state_idx] += 1
        R[state_idx, action_idx] = reward

discount_factor = 0.95

# Policy Evaluation
def policy_evaluation(policy, P, R, discount_factor, theta=1e-6):
    V = np.zeros(num_states) # Initialize value function
    while True:
        delta = 0
        for state_idx in range(num_states): # Iterate over all states
            v = V[state_idx] # Store the old value function
            V[state_idx] = sum(policy[state_idx, action_idx] *  # Calculate the new value function
                               sum(P[state_idx, action_idx, next_state_idx] *  # Calculate the expected value
                                   (R[state_idx, action_idx] + discount_factor * V[next_state_idx]) # Calculate the expected reward
                                   for next_state_idx in range(num_states)) # Bellman expectation
                               for action_idx in range(num_actions)) # Sum over all actions
            delta = max(delta, abs(v - V[state_idx])) # Calculate the maximum change in value function
        if delta < theta:
            break
    return V

# Policy Improvement
def policy_improvement(policy, V, P, R, discount_factor):
    policy_stable = True
    new_policy = np.zeros((num_states, num_actions))
    
    for state_idx in range(num_states):
        old_action = np.argmax(policy[state_idx])
        action_values = np.zeros(num_actions)
        
        for action_idx in range(num_actions):
            action_values[action_idx] = sum(P[state_idx, action_idx, next_state_idx] *
                                            (R[state_idx, action_idx] + discount_factor * V[next_state_idx])
                                            for next_state_idx in range(num_states))
        
        best_action = np.argmax(action_values)
        new_policy[state_idx] = np.eye(num_actions)[best_action]
        
        if old_action != best_action:
            policy_stable = False
    
    return new_policy, policy_stable

# Policy Iteration
def policy_iteration(P, R, discount_factor):
    policy = np.ones((num_states, num_actions)) / num_actions
    while True:
        V = policy_evaluation(policy, P, R, discount_factor)
        policy, policy_stable = policy_improvement(policy, V, P, R, discount_factor)
        if policy_stable:
            break
    return policy, V

optimal_policy, optimal_value_function = policy_iteration(P, R, discount_factor)

print("\nOptimal Value Function (Policy Iteration):")
for state, value in zip(state_space, optimal_value_function):
    print(f"Value of state {state}: {value:.2f}")

print("\nOptimal Policy (Policy Iteration):")
for state, actions in zip(state_space, optimal_policy):
    action_idx = np.argmax(actions)
    action = action_space[action_idx]
    print(f"State {state}: {action}")

# Value Iteration
def value_iteration(P, R, discount_factor, theta=1e-6):
    V = np.zeros(num_states)
    while True:
        delta = 0
        for state_idx in range(num_states):
            v = V[state_idx]
            action_values = np.zeros(num_actions)
            for action_idx in range(num_actions):
                action_values[action_idx] = sum(P[state_idx, action_idx, next_state_idx] *
                                                (R[state_idx, action_idx] + discount_factor * V[next_state_idx])
                                                for next_state_idx in range(num_states))
            V[state_idx] = np.max(action_values)
            delta = max(delta, abs(v - V[state_idx]))
        if delta < theta:
            break
    return V

def value_iteration_with_policy(P, R, discount_factor, theta=1e-6):
    V = value_iteration(P, R, discount_factor, theta)
    
    # Derive the optimal policy from the optimal value function
    optimal_policy = np.zeros((num_states, num_actions))
    for state_idx in range(num_states):
        action_values = np.zeros(num_actions)
        for action_idx in range(num_actions):
            action_values[action_idx] = sum(P[state_idx, action_idx, next_state_idx] *
                                            (R[state_idx, action_idx] + discount_factor * V[next_state_idx])
                                            for next_state_idx in range(num_states))
        best_action = np.argmax(action_values)
        optimal_policy[state_idx] = np.eye(num_actions)[best_action]
    
    return optimal_policy, V

optimal_policy_vi, optimal_value_function_vi = value_iteration_with_policy(P, R, discount_factor)

print("\nOptimal Value Function (Value Iteration):")
for state, value in zip(state_space, optimal_value_function_vi):
    print(f"Value of state {state}: {value:.2f}")

print("\nOptimal Policy (Value Iteration):")
for state, actions in zip(state_space, optimal_policy_vi):
    action_idx = np.argmax(actions)
    action = action_space[action_idx]
    print(f"State {state}: {action}")



Optimal Value Function (Policy Iteration):
Value of state (0, 0): 26.73
Value of state (0, 1): 28.14
Value of state (0, 2): 26.73
Value of state (0, 3): 25.64
Value of state (0, 4): 24.36
Value of state (1, 0): 25.40
Value of state (1, 1): 26.73
Value of state (1, 2): 25.40
Value of state (1, 3): 24.36
Value of state (1, 4): 23.14
Value of state (2, 0): 24.13
Value of state (2, 1): 25.40
Value of state (2, 2): 24.13
Value of state (2, 3): 23.14
Value of state (2, 4): 21.98
Value of state (3, 0): 22.92
Value of state (3, 1): 24.13
Value of state (3, 2): 22.92
Value of state (3, 3): 21.98
Value of state (3, 4): 20.88
Value of state (4, 0): 21.77
Value of state (4, 1): 22.92
Value of state (4, 2): 21.77
Value of state (4, 3): 20.88
Value of state (4, 4): 19.84

Optimal Policy (Policy Iteration):
State (0, 0): right
State (0, 1): up
State (0, 2): left
State (0, 3): up
State (0, 4): left
State (1, 0): right
State (1, 1): up
State (1, 2): up
State (1, 3): up
State (1, 4): up
State (2, 0): r

Part 2, implement the Monte Carlo method with exploring starts and without exploring starts, use policy iteration to determine a suitable policy 

In [8]:
import numpy as np

# Define the gridworld environment
state_space = [(i, j) for i in range(5) for j in range(5)]
action_space = ['up', 'down', 'left', 'right']
num_states = len(state_space)
num_actions = len(action_space)

# Map states to indices and vice versa
state_to_idx = {state: idx for idx, state in enumerate(state_space)}
idx_to_state = {idx: state for state, idx in state_to_idx.items()}

# Define the special states
blue = (0, 1)
green = (0, 3)
red = (1, 3)
yellow = (4, 3)
terminal_states = [(3, 0), (3, 1)]

# Define the transition probabilities and rewards
P = np.zeros((num_states, num_actions, num_states))
R = np.zeros((num_states, num_actions))

def step(state, action):
    if state in terminal_states:
        return state, 0  # Terminal state, no reward

    i, j = state
    if action == 'up':
        next_state = (max(i - 1, 0), j)
    elif action == 'down':
        next_state = (min(i + 1, 4), j)
    elif action == 'left':
        next_state = (i, max(j - 1, 0))
    elif action == 'right':
        next_state = (i, min(j + 1, 4))
    else:
        next_state = state

    # Special states
    if state == blue:
        reward = 5
        next_state = red
    elif state == green:
        reward = 2.5
        next_state = yellow if np.random.rand() < 0.5 else red
    elif next_state == state:
        reward = -0.5  # Stepping off the grid
    else:
        reward = -0.2  # Normal movement between white squares

    return next_state, reward

# Fill in the transition matrix P and reward vector R
for state in state_space:
    state_idx = state_to_idx[state]
    for action_idx, action in enumerate(action_space):
        next_state, reward = step(state, action)
        next_state_idx = state_to_idx[next_state]
        P[state_idx, action_idx, next_state_idx] += 1
        R[state_idx, action_idx] = reward

# Monte Carlo with Exploring Starts
def mc_exploring_starts(P, R, discount_factor, num_episodes=50000):
    policy = np.ones((num_states, num_actions)) / num_actions
    returns = np.zeros((num_states, num_actions))
    returns_count = np.zeros((num_states, num_actions))
    Q = np.zeros((num_states, num_actions))

    for episode in range(num_episodes):
        state = state_space[np.random.choice(num_states)]
        action = np.random.choice(num_actions)
        episode_states_actions_rewards = []

        # Generate an episode
        while state not in terminal_states:
            state_idx = state_to_idx[state]
            next_state, reward = step(state, action_space[action])
            next_state_idx = state_to_idx[next_state]
            episode_states_actions_rewards.append((state_idx, action, reward))
            state = next_state
            action = np.random.choice(num_actions)

        # Calculate returns
        G = 0
        for state_idx, action, reward in reversed(episode_states_actions_rewards):
            G = discount_factor * G + reward
            if (state_idx, action) not in [(x[0], x[1]) for x in episode_states_actions_rewards[:-1]]:
                returns[state_idx, action] += G
                returns_count[state_idx, action] += 1
                Q[state_idx, action] = returns[state_idx, action] / returns_count[state_idx, action]
                policy[state_idx] = np.eye(num_actions)[np.argmax(Q[state_idx])]

    return policy

optimal_policy_mc_es = mc_exploring_starts(P, R, discount_factor)

print("\nOptimal Policy (MC with Exploring Starts):")
for state, actions in zip(state_space, optimal_policy_mc_es):
    action_idx = np.argmax(actions)
    action = action_space[action_idx]
    print(f"State {state}: {action}")





Optimal Policy (MC with Exploring Starts):
State (0, 0): up
State (0, 1): up
State (0, 2): up
State (0, 3): up
State (0, 4): up
State (1, 0): up
State (1, 1): up
State (1, 2): up
State (1, 3): up
State (1, 4): up
State (2, 0): up
State (2, 1): up
State (2, 2): up
State (2, 3): up
State (2, 4): up
State (3, 0): up
State (3, 1): up
State (3, 2): up
State (3, 3): up
State (3, 4): up
State (4, 0): down
State (4, 1): down
State (4, 2): up
State (4, 3): up
State (4, 4): up


In [9]:
# Monte Carlo with epsilon-soft policy
def mc_epsilon_soft(P, R, discount_factor, epsilon=0.1, num_episodes=50000):
    policy = np.ones((num_states, num_actions)) / num_actions
    returns = np.zeros((num_states, num_actions))
    returns_count = np.zeros((num_states, num_actions))
    Q = np.zeros((num_states, num_actions))

    for episode in range(num_episodes):
        state = state_space[np.random.choice(num_states)]
        episode_states_actions_rewards = []

        # Generate an episode
        while state not in terminal_states:
            state_idx = state_to_idx[state]
            action = np.random.choice(num_actions, p=policy[state_idx])
            next_state, reward = step(state, action_space[action])
            next_state_idx = state_to_idx[next_state]
            episode_states_actions_rewards.append((state_idx, action, reward))
            state = next_state

        # Calculate returns
        G = 0
        for state_idx, action, reward in reversed(episode_states_actions_rewards):
            G = discount_factor * G + reward
            if (state_idx, action) not in [(x[0], x[1]) for x in episode_states_actions_rewards[:-1]]:
                returns[state_idx, action] += G
                returns_count[state_idx, action] += 1
                Q[state_idx, action] = returns[state_idx, action] / returns_count[state_idx, action]
                best_action = np.argmax(Q[state_idx])
                for a in range(num_actions):
                    if a == best_action:
                        policy[state_idx, a] = 1 - epsilon + (epsilon / num_actions)
                    else:
                        policy[state_idx, a] = epsilon / num_actions

    return policy

optimal_policy_mc_epsilon_soft = mc_epsilon_soft(P, R, discount_factor)

print("\nOptimal Policy (MC with epsilon-soft):")
for state, actions in zip(state_space, optimal_policy_mc_epsilon_soft):
    action_idx = np.argmax(actions)
    action = action_space[action_idx]
    print(f"State {state}: {action}")

import random

def permute_special_states():
    if random.random() < 0.1:
        global blue, green
        blue, green = green, blue

# Policy Iteration with Permuting Locations
def policy_iteration_with_permutations(P, R, discount_factor):
    policy = np.ones((num_states, num_actions)) / num_actions
    while True:
        permute_special_states()
        V = policy_evaluation(policy, P, R, discount_factor)
        policy, policy_stable = policy_improvement(policy, V, P, R, discount_factor)
        if policy_stable:
            break
    return policy, V

optimal_policy_permuted, optimal_value_function_permuted = policy_iteration_with_permutations(P, R, discount_factor)

print("\nOptimal Value Function (Policy Iteration with Permutations):")
for state, value in zip(state_space, optimal_value_function_permuted):
    print(f"Value of state {state}: {value:.2f}")

print("\nOptimal Policy (Policy Iteration with Permutations):")
for state, actions in zip(state_space, optimal_policy_permuted):
    action_idx = np.argmax(actions)
    action = action_space[action_idx]
    print(f"State {state}: {action}")



Optimal Policy (MC with epsilon-soft):
State (0, 0): up
State (0, 1): up
State (0, 2): up
State (0, 3): up
State (0, 4): up
State (1, 0): up
State (1, 1): up
State (1, 2): up
State (1, 3): up
State (1, 4): up
State (2, 0): up
State (2, 1): up
State (2, 2): up
State (2, 3): up
State (2, 4): up
State (3, 0): up
State (3, 1): up
State (3, 2): up
State (3, 3): up
State (3, 4): up
State (4, 0): down
State (4, 1): down
State (4, 2): up
State (4, 3): up
State (4, 4): up

Optimal Value Function (Policy Iteration with Permutations):
Value of state (0, 0): 24.68
Value of state (0, 1): 26.19
Value of state (0, 2): 24.68
Value of state (0, 3): 23.69
Value of state (0, 4): 22.31
Value of state (1, 0): 23.25
Value of state (1, 1): 24.68
Value of state (1, 2): 23.25
Value of state (1, 3): 22.31
Value of state (1, 4): 20.99
Value of state (2, 0): 21.89
Value of state (2, 1): 23.25
Value of state (2, 2): 21.89
Value of state (2, 3): 20.99
Value of state (2, 4): 19.74
Value of state (3, 0): 0.00
Value 